### question 5

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

# ------------------------------------------------------------
# 2-a.  Load the data  (file: three columns – x1, x2, label)
data = np.loadtxt('data0.txt')
X, y = data[:, :2], data[:, 2].astype(int)
k = len(np.unique(y))          # number of classes (4)

# ------------------------------------------------------------
# 2-b.  Multiclass-Perceptron functions
def mc_predict(W, b, x):
    """Return argmax_j  (w_j · x + b_j)."""
    scores = W @ x + b
    return scores.argmax()

def fit_multiclass_perceptron(X, y, epochs=30, seed=42):
    """Train weights (W, b) for k classes in d dims."""
    rng = np.random.RandomState(seed)
    X, y = shuffle(X, y, random_state=rng)

    k = y.max() + 1            # assumes classes = {0,…,k-1}
    d = X.shape[1]
    W = np.zeros((k, d))
    b = np.zeros(k)

    for _ in range(epochs):
        mistakes = 0
        for x_i, y_i in zip(X, y):
            y_hat = mc_predict(W, b, x_i)
            if y_hat != y_i:
                W[y_i] += x_i
                b[y_i] += 1
                W[y_hat] -= x_i
                b[y_hat] -= 1
                mistakes += 1
        if mistakes == 0:
            break  # converged
    return W, b

W, b = fit_multiclass_perceptron(X, y)

# ------------------------------------------------------------
# 2-c.  Visualise decision regions
def plot_regions(X, y, W, b):
    # colour map for four classes
    cmap = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red']

    # mesh grid covering the data span
    margin = 0.5
    x_min, x_max = X[:,0].min() - margin, X[:,0].max() + margin
    y_min, y_max = X[:,1].min() - margin, X[:,1].max() + margin
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                         np.linspace(y_min, y_max, 300))
    grid = np.c_[xx.ravel(), yy.ravel()]
    preds = np.array([mc_predict(W, b, p) for p in grid]).reshape(xx.shape)

    plt.figure(figsize=(6,5))
    plt.contourf(xx, yy, preds, alpha=0.25, levels=np.arange(k+1)-0.5,
                 colors=cmap)
    # scatter original points
    for cls in range(k):
        idx = y == cls
        plt.scatter(X[idx,0], X[idx,1], c=cmap[cls], edgecolor='k',
                    label=f'class {cls}', s=40)
    plt.xlabel('$x_1$'); plt.ylabel('$x_2$')
    plt.title('Multiclass Perceptron decision regions')
    plt.legend(); plt.tight_layout(); plt.show()

plot_regions(X, y, W, b)


In [ ]:
'''
create binary_perceptron function
takes as input parameters w, b of a linear classifier as well as a data point
x, and returns the label for that point: sign(w · x + b). The label is either +1 or -1.
'''
def binary_peceptron(w, b, x):
    if np.dot(w, x) + b >= 0:
        label = 1
    else:
        label = -1
    return(label)
    
'''
create fit_binary_perceptron function
takes as input an array of data points and an array of labels (where each
label is +1 or -1), and runs the Perceptron algorithm to learn a linear classifier w, b. The
algorithm should begin by randomly permuting the data points.
'''
def fit_binary_perceptron(x, y, track_updates, set_seed):
    if set_seed:
        x, y = shuffle(x, y,random_state=42)
    else:
        x, y = shuffle(x, y)
    w = np.zeros(x.shape[1])
    b = 0
    updates = 0
    max_updates = 1000

    def make_prediction(w, b, x, y, updates):
        error = False    
        for xi, yi in zip(x, y):
            if binary_perceptron(w, b, xi) != yi:
                w += yi * xi
                b += yi
                updates += 1
                error = True  
        if updates <= max_updates:
            if error:
                return make_prediction(w, b, x, y, updates) 
            else:
                return (w, b, updates)
        else:
            print("Did not converge after {max_iterations} iterations")
    if track_updates:
        w, b, updates = make_prediction(w, b, x, y, updates)
        return (w, b, updates)
    else:
        w, b, updates = make_prediction(w, b, x, y, updates)
        return (w, b)
